データの読み込み

In [ ]:
import pandas as pd
import datetime
from datetime import datetime as dt
df = pd.read_csv('../dataset/data.csv', encoding='shift_jis')

target_list = ['ShopC','SalesDate','ProductC','RecAve','SalesVol']
target_df = df[target_list]
target_df

SalesDateをdatetimeに変更し、年・月・日に分解

In [ ]:

date_list = []
year_list = []
month_list = []
day_list = []
season_list = []
for index, i in enumerate(target_df['SalesDate'].values):
    date = str(i)
    year = int(date[0:4])
    month = int(date[4:6])
    day = int(date[6:])

    date_list.append(datetime.date(year, month, day))
    year_list.append(year)
    month_list.append(month)
    day_list.append(day)

target_df['SalesDate'] = date_list
target_df['Year'] = year_list
target_df['Month'] = month_list
target_df['Day'] = day_list
target_df

datasetの1次元はProductC

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

datasetの2次元はShopC

[617, 651, 833, 921, 922, 934, 937, 1190, 1250, 1261, 1236, 671, 672]



In [ ]:
product_code = list(range(1,10+1))
store_code = [617,651,833,921,922,934,937,1190,1250,1261,1236,671,672]

dataset = [[None]*len(store_code) for _ in range(len(product_code))]

for i in range(len(store_code)):
    store_df = target_df[target_df['ShopC'] == store_code[i]]
    for j in range(len(product_code)):
        dataset[j][i] = store_df[store_df['ProductC'] == product_code[j]]

In [ ]:
dataset[0][0]

データの参照をしたいため、新しくデータフレームを作成

In [ ]:
# weighted_df = pd.read_csv('../dataset/weighted.csv', encoding='shift_jis')

# date_list = []
# for i in weighted_df['SalesDate'].values:
#     date = str(i)
#     date_list.append(datetime.date(int(date[0:4]), int(date[5:7]), int(date[8:])))
# weighted_df['SalesDate'] = date_list
# weighted_df = weighted_df.set_index('SalesDate')
# weighted_df


In [ ]:
day_df = pd.read_csv('../dataset/salesdate.csv', encoding='shift_jis')
date_list = []
for i in day_df['SalesDate'].values:
    date = str(i)
    date_list.append(datetime.date(int(date[0:4]), int(date[5:7]), int(date[8:])))
day_df['SalesDate'] = date_list
day_df = day_df.set_index('SalesDate')


weather_df = pd.read_csv('../dataset/weather.csv', encoding='shift_jis')
date_list = []
for i in weather_df['SalesDate'].values:
    date = str(i)
    date_list.append(datetime.date(int(date[0:4]), int(date[5:7]), int(date[8:])))
weather_df['SalesDate'] = date_list
weather_df = weather_df.set_index('SalesDate')


new_df = pd.concat([day_df, weather_df], axis = 1)
new_df

データの欠損値を埋める

In [ ]:
import numpy as np
indexs = len(target_df) + 1
idx = 0
for i in range(len(store_code)):
    for j in range(len(product_code)):
        data = dataset[j][i]
        max_value = data['RecAve'].max()
        if str(max_value) == "nan":
            max_value = 0
        for k in day_df.index:
            if not (k in data['SalesDate'].values):
                str_k = str(k)
                yaer, month, day = int(str_k[:4]), int(str_k[5:7]), int(str_k[8:])
                data.loc[indexs] = [store_code[i], k, product_code[j], max_value, 0, 0, yaer, month, day]
                indexs += 1
        data = data.drop_duplicates(subset=['SalesDate']).sort_values('SalesDate').set_index('SalesDate')
        data = pd.concat([data, new_df], axis = 1).reset_index()
        data.index = np.arange(idx, len(data)+idx)
        idx += len(data)
        dataset[j][i] = data.rename(columns={'index': 'SalesDate'})

In [ ]:
dataset[0][0]

In [ ]:
save_columns = ['SalesDate', 'ShopC', 'ProductC', 'RecAve', 'SalesCate', 'Year', 'Month', 'Day',
                'Season', 'WeekDay', 'Holiday', 'TempAve', 'PreAmo', 'WindAve', 'VapAve', 'CloAve']
save_data = pd.DataFrame(columns = save_columns)
for i in range(len(store_code)):
    for j in range(len(product_code)):
        save_data = pd.concat([save_data, dataset[j][i]], axis = 0)
save_data

CSVデータとして保存

In [ ]:
save_data.to_csv('../dataset/dataset.csv', encoding = 'shift_jis',index=None)